In [1]:
import pandas as pd
import numpy as np
import openpyxl as px
import matplotlib.pyplot as plt
from curl_cffi import requests
import seaborn as sns
import yfinance as yf
from datetime import datetime, timedelta

In [2]:
import yfinance as yf
import requests
import pandas as pd
from datetime import datetime, timedelta

# Session for requests reuse
session = requests.Session()

portfolio = ['AAPL', 'JPM', 'JNJ', 'PG', 'XOM', 'MSFT', 'AMZN']
bench = ['SPY', 'DIA', 'IWM']

end_date = datetime.now()
start_date = end_date - timedelta(days=365*10)

# Download portfolio data
portfolio_data = {}
for ticker in portfolio:
    ticker_obj = yf.Ticker(ticker, session=session)
    hist = ticker_obj.history(start=start_date, end=end_date)
    portfolio_data[ticker] = hist['Close']  # or 'Adj Close'

portfolio_df = pd.DataFrame(portfolio_data)

# Download benchmark data
bench_data = {}
for ticker in bench:
    ticker_obj = yf.Ticker(ticker, session=session)
    hist = ticker_obj.history(start=start_date, end=end_date)
    bench_data[ticker] = hist['Close']

bench_df = pd.DataFrame(bench_data)

print("Portfolio data shape:", portfolio_df.shape)
print("Benchmarks data shape:", bench_df.shape)


YFRateLimitError: Too Many Requests. Rate limited. Try after a while.

In [3]:
#Portfolio Set-up
portfolio = ['AAPL', 'JPM', 'JNJ', 'PG', 'XOM', 'MSFT', 'AMZN']
bench = ['SPY', 'DIA', 'IWM']
tickers = portfolio + bench


In [32]:
# Create session with impersonation (for curl_cffi)
session = requests.Session(impersonate="chrome")

In [4]:
# Download historical data (10 years of data)
end_date = datetime.now()
start_date = end_date - timedelta(days=365*10)

# Download data
data = yf.download(tickers, period='10y', group_by='ticker') 


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  10 of 10 completed

10 Failed downloads:
['JNJ', 'MSFT', 'IWM', 'SPY', 'DIA', 'AMZN', 'JPM', 'PG', 'AAPL', 'XOM']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


In [5]:
returns = data.pct_change

In [6]:
data.columns

MultiIndex([( 'JNJ',      'Open'),
            ( 'JNJ',      'High'),
            ( 'JNJ',       'Low'),
            ( 'JNJ',     'Close'),
            ( 'JNJ', 'Adj Close'),
            ( 'JNJ',    'Volume'),
            ('MSFT',      'Open'),
            ('MSFT',      'High'),
            ('MSFT',       'Low'),
            ('MSFT',     'Close'),
            ('MSFT', 'Adj Close'),
            ('MSFT',    'Volume'),
            ( 'IWM',      'Open'),
            ( 'IWM',      'High'),
            ( 'IWM',       'Low'),
            ( 'IWM',     'Close'),
            ( 'IWM', 'Adj Close'),
            ( 'IWM',    'Volume'),
            ( 'SPY',      'Open'),
            ( 'SPY',      'High'),
            ( 'SPY',       'Low'),
            ( 'SPY',     'Close'),
            ( 'SPY', 'Adj Close'),
            ( 'SPY',    'Volume'),
            ( 'DIA',      'Open'),
            ( 'DIA',      'High'),
            ( 'DIA',       'Low'),
            ( 'DIA',     'Close'),
            ( 'DIA',

In [7]:
tickers = list(map(str.upper, tickers))
bench = list(map(str.upper, bench))
portfolio = list(map(str.upper, portfolio))

In [8]:
table = pd.DataFrame(index=tickers)


In [9]:
# 1. Portfolio Weight (Equally Weighted)
table['Portfolio Weight'] = 1 / len(tickers)


In [10]:
# 2. Annualized Volatility (trailing 3 months = ~63 trading days)
vol_window = 63
annualization_factor = 252 ** 0.5
table['Annualized Volatility'] = returns[tickers].iloc[-vol_window:].std() * annualization_factor


TypeError: 'method' object is not subscriptable

In [ ]:
# 3-5. Beta against SPY, IWM, DIA (trailing 12 months = ~252 trading days)
beta_window = 252
cov_matrix = returns.iloc[-beta_window:].cov()
var_bench = returns.iloc[-beta_window:][bench].var()

for etf in bench:
    table_1[f'Beta_{etf}'] = cov_matrix.loc[tickers, etf] / var_bench[etf]

In [ ]:
# 6-7. Average and Maximum Weekly Drawdowns (based on trailing 52-week data = 252 trading days)
drawdown_window = 252
high_5d = hist_data[tickers].iloc[-drawdown_window:].rolling(window=5).max()
low_5d = hist_data[tickers].iloc[-drawdown_window:].rolling(window=5).min()
weekly_drawdown = (low_5d - high_5d) / high_5d

table_1['Average Weekly Drawdown'] = weekly_drawdown.mean()
table_1['Maximum Weekly Drawdown'] = weekly_drawdown.min()


In [ ]:
# 8-9. 10-Year Total Return and Annualized Return
# Assumes 10 years of daily data (~2520 days)
total_return = hist_data[tickers].iloc[-1] / hist_data[tickers].iloc[0] - 1
annualized_return = (1 + total_return) ** (1/10) - 1

table_1['10Y Total Return'] = total_return
table_1['10Y Annualized Return'] = annualized_return


In [ ]:
# Reset index to include Ticker as a column
table_1.reset_index(inplace=True)

# Reorder columns for final output
ordered_cols = [
    'Ticker',
    'Portfolio Weight',
    'Annualized Volatility',
    'Beta_SPY',
    'Beta_IWM',
    'Beta_DIA',
    'Average Weekly Drawdown',
    'Maximum Weekly Drawdown',
    '10Y Total Return',
    '10Y Annualized Return'
]
table_1 = table_1[ordered_cols]